# Superdense Coding Protocol - Problem 2 (Part A)
This notebook implements the superdense coding scheme, a quantum communication protocol that leverages entanglement to send two classical bits of information using only a single qubit.

### Setup
Importing the required Qiskit components for quantum circuit design, simulation, and histogram plotting.

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram

### 1. Preparing the Entangled Resource
Both parties initially share a Bell state $|\Phi^+\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$. Qubit 0 belongs to Alice and qubit 1 belongs to Bob. The Bell state is constructed by applying H on qubit 0 followed by CNOT targeting qubit 1.

In [ ]:
q = QuantumRegister(2, 'q')
c = ClassicalRegister(2, 'c')
dense_circ = QuantumCircuit(q, c)

dense_circ.h(0)
dense_circ.cx(0, 1)
dense_circ.barrier()

### 2. Encoding the Two-Bit Message
The classical message to transmit is **'11'**. Following the standard superdense coding lookup table:
- '00' $\rightarrow$ Identity (no gate)
- '01' $\rightarrow$ X gate
- '10' $\rightarrow$ Z gate
- '11' $\rightarrow$ X gate then Z gate

Since our message is '11', Alice applies X followed by Z to her qubit.

In [ ]:
msg = '11'

dense_circ.x(0)
dense_circ.z(0)
dense_circ.barrier()

### 3. Bob's Decoding Procedure
After receiving Alice's qubit, Bob reverses the entanglement by applying CNOT (qubit 0 controlling qubit 1) and Hadamard on qubit 0. This maps each of the four possible Bell states back to a unique two-qubit computational basis state, revealing the encoded message.

In [ ]:
dense_circ.cx(0, 1)
dense_circ.h(0)
dense_circ.barrier()

### 4. Measurement
Both qubits are measured in the computational basis. The two-bit measurement outcome should exactly match the message Alice intended to send.

In [ ]:
dense_circ.measure([0, 1], [0, 1])

dense_circ.draw('mpl')

### 5. Execution and Verification
Running 1000 shots on the Aer simulator. A deterministic result of '11' validates that the superdense coding protocol successfully communicated two bits of classical data through one qubit.

In [ ]:
simulator = Aer.get_backend('aer_simulator')
execution = simulator.run(dense_circ, shots=1000)
results = execution.result().get_counts()

print(f"Message sent: {msg}")
print(f"Measurement counts: {results}")

plot_histogram(results)